In [1]:
import spacy
from spacy.tokens import DocBin
import json

In [2]:
!pip install spacy_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 46.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:
      Successfully uninstalled transformers-4.47.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.3.1 requires transformers<5.0.0,>=4.41.0, but you hav

In [3]:
!python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency


/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [5]:
# Step 1: Load the JSON annotations
def load_annotations(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    print(f"Loaded {len(data)} annotations.")
    return data

# Step 2: Convert to SpaCy format
def create_spacy_data(data, output_path):
    nlp = spacy.blank("en")  # Create a blank English NLP pipeline
    doc_bin = DocBin()  # SpaCy container for serialized documents

    for item in data:
        text, annotations = item[0], item[1]
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annotations.get("entities", []):
            span = doc.char_span(start, end, label=label)
            if span is None:
                print(f"Skipping invalid span: {text[start:end]}")
            else:
                ents.append(span)
        doc.ents = ents
        doc_bin.add(doc)
    doc_bin.to_disk(output_path)
    print(f"Saved data to {output_path}")

# def create_spacy_data(data, output_path):
#     nlp = spacy.blank("en")  # Create a blank English model
#     doc_bin = DocBin()  # Container for serialized training examples

#     for item in data:
#         text = item["text"]
#         annotations = item["entities"]

#         doc = nlp.make_doc(text)
#         ents = []

#         # Iterate through each entity and create a span
#         for start, end, label in annotations:
#             span = doc.char_span(start, end, label=label)

#             if span is None:
#                 # Check if the span is invalid and try to handle it
#                 print(f"Invalid span: {text[start:end]} (start: {start}, end: {end})")
#                 # Optional: If necessary, you can try adjusting the span here
#             else:
#                 ents.append(span)

#         doc.ents = ents
#         doc_bin.add(doc)

#     doc_bin.to_disk(output_path)
#     print(f"Saved data to {output_path}")

# Step 3: Train the NER model
def train_ner(train_data_path, dev_data_path, output_dir):
    !python -m spacy train config.cfg \
        --paths.train {train_data_path} \
        --paths.dev {dev_data_path} \
        --output {output_dir}

# Main function
def main():
    # File paths
    input_json = "/content/edited_annotations_v2.json"
    train_output = "/content/train_data.spacy"
    dev_output = "/content/dev_data.spacy"
    model_output = "/content/output"

    # Load and process the annotations
    data = load_annotations(input_json)
    train_data = data[:int(0.8 * len(data))]  # 80% for training
    dev_data = data[int(0.8 * len(data)):]    # 20% for validation

    create_spacy_data(train_data, train_output)
    create_spacy_data(dev_data, dev_output)

    # Train the NER model
    train_ner(train_output, dev_output, model_output)

if __name__ == "__main__":
    main()


Loaded 101 annotations.
Skipping invalid span: FLUID RECOVERY
Skipping invalid span: FLUID RECOVERY
Saved data to /content/train_data.spacy
Skipping invalid span: FLUID RECOVERY
Saved data to /content/dev_data.spacy
✔ Created output directory: /content/output
ℹ Saving to output directory: /content/output
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
✔ Initialized pipeline

============================

In [9]:
import spacy

# Load the trained model
nlp = spacy.load("/content/output/model-best")

# Test on a new text
test_text = "RESERVOIR FLUID STUDY TABLE A1 ENCANA CORPORATION - SAXXON WELL 16-05-062-24W5M - - RECOMBINED SAMPLE CONSTANT VOLUME DEPLETION COMPOSITIONAL ANALYSIS OF RESERVOIR FLUID Boiling Point Component Chemical Mole Mass Calculated Properties (F) Name Symbol Fraction Fraction -320.4 Nitrogen N 0.0021 0.0015 Total Sample 2 -109.3 Carbon Dioxide CO 0.0035 0.0038 2 -76.6 Hydrogen Sulphide HS 0.0000 0.0000 Molecular Weight 40.42 2 -259.1 Methane C 0.6048 0.2402 Density (g/cc) 0.4044 1 -128.0 Ethane C 0.1317 0.0980 2 -44.0 Propane C 0.0604 0.0659 C Fraction 3 6+ 10.9 i-Butane i-C 0.0125 0.0179 4 30.9 n-Butane n-C 0.0279 0.0401 Molecular Weight 150.40 4 82.0 i-Pentane i-C 0.0120 0.0214 Mole Fraction 0.1303 5 97.0 n-Pentane n-C 0.0148 0.0263 Density (g/cc) 0.8022 5 97 - 156 Hexanes C 0.0184 0.0393 6 156 - 208.9 Heptanes C 0.0138 0.0342 C Fraction 7 7+ 208.9 - 258.1 Octanes C 0.0158 0.0445 8 258.1 - 303.1 Nonanes C 0.0120 0.0381 Molecular Weight 159.93 9 303.1 - 345 Decanes C 0.0086 0.0304 Mole Fraction 0.1114 10 345 - 385 Undecanes C 0.0076 0.0276 Density (g/cc) 0.8146 11 385 - 419 Dodecanes C 0.0057 0.0229 12 419 - 455 Tridecanes C 0.0054 0.0233 C Fraction 13 12+ 455 - 486 Tetradecanes C 0.0041 0.0192 14 486 - 519.1 Pentadecanes C 0.0030 0.0154 Molecular Weight 253.63 15 519.1 - 550 Hexadecanes C 0.0024 0.0131 Mole Fraction 0.0364 16 550 - 557 Heptadecanes C 0.0020 0.0118 Density (g/cc) 0.8654 17 557 - 603 Octadecanes C 0.0018 0.0114 18 603 - 626 Nonadecanes C 0.0016 0.0104 19 626 - 651.9 Eicosanes C 0.0012 0.0084 20 651.9 - 675 Heneicosanes C 0.0011 0.0076 21 675 - 696.9 Docosanes C 0.0009 0.0069 22 696.9 - 716 Tricosanes C 0.0008 0.0062 23 716 - 736 Tetracosanes C 0.0007 0.0058 24 736 - 755.1 Pentacosanes C 0.0006 0.0053 25 755.1 - 774 Hexacosanes C 0.0006 0.0049 Recombination Parameters 26 774.1 - 792 Heptacosanes C 0.0005 0.0043 27 792.1 - 809.1 Octacosanes C 0.0004 0.0038 Gas-Oil Ratio (cc/cc) 715.33 28 809.1 - 826 Nonacosanes C 0.0004 0.0035 Dead Oil Density (g/cc) 0.7595 29 Above 826 Tricontanes Plus C 0.0032 0.0439 Dead Oil MW (g/mol) 134.87 30+ NAPHTHENES 120.0 Cyclopentane CH 0.0004 0.0007 5 10 162.0 Methylcyclopentane CH 0.0020 0.0042 6 12 178.0 Cyclohexane CH 0.0024 0.0050 6 12 214.0 Methylcyclohexane CH 0.0057 0.0137 7 14 AROMATICS 176.0 Benzene C 6H 6 0.0004 0.0007 231.1 Toluene CH 0.0018 0.0041 7 8 277 - 282 Ethylbenzene & p,m-Xylene CH 0.0017 0.0045 8 10 291.9 o-Xylene CH 0.0012 0.0031 8 10 336.0 1, 2, 4-Trimethylbenzene CH 0.0021 0.0064 9 12 Total 1.0000 1.0000 Note: Physical Properties calculated based GPA 2145-00 physical constants 27"
test_text = "ENCANA CORPORATION TABLE B27 ENCANA CORPORATION - WILLESDEN GREEN WELL 03-06-043-07 W5 - DUVERNAY - SAMPLE W3A7578 SAMPLE VALIDATION COMPOSITIONAL ANALYSIS OF FLASHED OIL Boiling Point Component Chemical Mole Mass Calculated Properties (F) Name Symbol Fraction Fraction -320.4 Nitrogen N 0.0000 0.0000 Total Sample 2 -109.3 Carbon Dioxide CO 0.0000 0.0000 2 -76.6 Hydrogen Sulphide HS 0.0000 0.0000 Molecular Weight 110.97 2 -259.1 Methane C 0.0000 0.0000 Density (g/cc) 0.7421 1 -128.0 Ethane C 0.0000 0.0000 2 -44.0 Propane C 0.0241 0.0096 C Fraction 3 6+ 10.9 i-Butane i-C 0.0256 0.0134 4 30.9 n-Butane n-C 0.0619 0.0324 Molecular Weight 125.57 4 82.0 i-Pentane i-C 0.0572 0.0372 Mole Fraction 0.7623 5 97.0 n-Pentane n-C 0.0689 0.0448 Density (g/cc) 0.7724 5 97 - 156 Hexanes C 0.1182 0.0918 6 156 - 208.9 Heptanes C 0.1012 0.0914 C Fraction 7 7+ 208.9 - 258.1 Octanes C 0.1172 0.1207 8 258.1 - 303.1 Nonanes C 0.0851 0.0983 Molecular Weight 132.98 9 303.1 - 345 Decanes C 0.0551 0.0707 Mole Fraction 0.6422 10 345 - 385 Undecanes C 0.0414 0.0548 Density (g/cc) 0.7844 11 385 - 419 Dodecanes C 0.0283 0.0411 12 419 - 455 Tridecanes C 0.0253 0.0399 C Fraction 13 12+ 455 - 486 Tetradecanes C 0.0174 0.0299 14 486 - 519.1 Pentadecanes C 0.0126 0.0233 Molecular Weight 208.26 15 519.1 - 550 Hexadecanes C 0.0084 0.0169 Mole Fraction 0.1228 16 550 - 557 Heptadecanes C 0.0069 0.0146 Density (g/cc) 0.8345 17 557 - 603 Octadecanes C 0.0055 0.0124 18 603 - 626 Nonadecanes C 0.0042 0.0100 C Fraction 19 30+ 626 - 651.9 Eicosanes C 0.0029 0.0072 20 651.9 - 675 Heneicosanes C 0.0024 0.0062 Molecular Weight 458.99 21 675 - 696.9 Docosanes C 0.0018 0.0050 Mole Fraction 0.0017 22 696.9 - 716 Tricosanes C 0.0014 0.0041 Density (g/cc) 0.9681 23 716 - 736 Tetracosanes C 0.0011 0.0034 24 736 - 755.1 Pentacosanes C 0.0009 0.0028 25 755.1 - 774 Hexacosanes C 0.0007 0.0023 26 774.1 - 792 Heptacosanes C 0.0005 0.0019 27 792.1 - 809.1 Octacosanes C 0.0004 0.0016 28 809.1 - 826 Nonacosanes C 0.0003 0.0013 29 Above 826 Tricontanes Plus C 0.0017 0.0070 30+ NAPHTHENES 0.20 120.0 Cyclopentane C 5H 10 0.0020 0.0012 0.18 162.0 Methylcyclopentane C 6H 12 0.0105 0.0080 0.16 178.0 Cyclohexane C 6H 12 0.0164 0.0125 0.14 214.0 Methylcyclohexane CH 0.0456 0.0404 7 14 0.12 0.10 AROMATICS 0.08 176.0 Benzene C 6H 6 0.0039 0.0027 0.06 231.1 Toluene CH 0.0187 0.0155 7 8 0.04 277 - 282 Ethylbenzene & p,m-Xylene CH 0.0109 0.0104 8 10 0.02 291.9 o-Xylene CH 0.0060 0.0057 8 10 0.00 336.0 1, 2, 4-Trimethylbenzene CH 0.0074 0.0081 9 12 Total 1.0000 1.0000 Note: Physical properties are calculated based on GPA 2145-00 physical constants GC ID: 2141 57 2N 2OC S2H 1C 2C 3C 4C 5C 6C 7C 8C 9C 01C 11C 21C 31C 41C 51C 61C 71C 81C 91C 02C 12C 22C 32C 42C 52C 62C 72C 82C 92C +03C"
doc = nlp(test_text)

# Print recognized entities
for ent in doc.ents:
    print(ent.text, ent.label_)


/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


SAMPLE W3A7578 SAMPLE
SAMPLE VALIDATION EXPERIMENT
COMPOSITIONAL ANALYSIS ANALYSIS
FLASHED OIL FLUID


In [6]:
!zip -r '/content/Duvernay_condensate_NER_v2' '/content/output/model-best'

  adding: content/output/model-best/ (stored 0%)
  adding: content/output/model-best/config.cfg (deflated 61%)
  adding: content/output/model-best/meta.json (deflated 64%)
  adding: content/output/model-best/tok2vec/ (stored 0%)
  adding: content/output/model-best/tok2vec/cfg (stored 0%)
  adding: content/output/model-best/tok2vec/model (deflated 8%)
  adding: content/output/model-best/vocab/ (stored 0%)
  adding: content/output/model-best/vocab/vectors (deflated 45%)
  adding: content/output/model-best/vocab/strings.json (deflated 75%)
  adding: content/output/model-best/vocab/lookups.bin (stored 0%)
  adding: content/output/model-best/vocab/vectors.cfg (stored 0%)
  adding: content/output/model-best/vocab/key2row (stored 0%)
  adding: content/output/model-best/ner/ (stored 0%)
  adding: content/output/model-best/ner/cfg (deflated 33%)
  adding: content/output/model-best/ner/moves (deflated 62%)
  adding: content/output/model-best/ner/model (deflated 8%)
  adding: content/output/model